In [3]:
# ====================================
# 1) Install dependencies
# ====================================
!pip install pandas python-dateutil openai matplotlib reportlab

# ====================================
# 2) Imports & Helper Functions
# ====================================
import os
import pandas as pd
import matplotlib.pyplot as plt
import json
from openai import OpenAI
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

# ---- Set API Key ----
os.environ["OPENAI_API_KEY"] = "sk"  # Replace with your API key

# ---- JSON extractor for OpenAI ----
def _extract_json(text: str):
    try:
        start = text.index("{")
        end = text.rindex("}") + 1
        return json.loads(text[start:end])
    except Exception as e:
        return {"error": f"Failed to parse JSON: {e}", "raw": text}

# ---- OpenAI ask function ----
def ask_openai(question: str, context: str, model="gpt-4o-mini"):
    client = OpenAI()
    system = (
        "You are a careful data analyst. "
        "You are given dataset context and must respond in valid JSON with keys: "
        '["answer","reasoning","assumptions","confidence","suggested_followups"].'
    )
    user = f"""DATA CONTEXT:
{context}

QUESTION:
{question}

Return valid JSON only.
"""

    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ],
        temperature=0.2
    )
    return _extract_json(resp.output_text)

# ====================================
# 3) Load datasets
# ====================================
sales_df = pd.read_csv("sales_data_large.csv")
students_df = pd.read_csv("student_marks_large.csv")

print("Sales dataset shape:", sales_df.shape)
print("Students dataset shape:", students_df.shape)

# ====================================
# 4) Data Cleaning
# ====================================
def clean_data(df):
    df = df.drop_duplicates()
    df = df.dropna(how='all')  # Remove fully empty rows
    df = df.fillna("Missing")  # Fill missing with placeholder
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()
    return df

sales_df = clean_data(sales_df)
students_df = clean_data(students_df)

# ====================================
# 5) Data Summary
# ====================================
sales_summary = sales_df.describe(include="all").transpose()
students_summary = students_df.describe(include="all").transpose()

display(sales_summary)
display(students_summary)

# ====================================
# 6) Combine datasets (optional)
# ====================================
combined_df = pd.concat(
    [sales_df.reset_index(drop=True), students_df.reset_index(drop=True)], 
    axis=1
)
print("Combined dataset shape:", combined_df.shape)

# ====================================
# 7) Visualizations
# ====================================
# Sales trend
if 'Date' in sales_df.columns and 'Sales_Amount' in sales_df.columns:
    sales_df['Date'] = pd.to_datetime(sales_df['Date'], errors='coerce')
    sales_df.groupby(sales_df['Date'].dt.to_period('M'))['Sales_Amount'].sum().plot(kind='line')
    plt.title("Monthly Sales Trend")
    plt.xlabel("Month")
    plt.ylabel("Total Sales")
    plt.show()

# Marks distribution
if 'Marks' in students_df.columns:
    students_df['Marks'].astype(float).plot(kind='hist', bins=10, edgecolor='black')
    plt.title("Student Marks Distribution")
    plt.xlabel("Marks")
    plt.ylabel("Frequency")
    plt.show()

# ====================================
# 8) OpenAI Analysis
# ====================================
context_sales = sales_summary.to_string()
context_students = students_summary.to_string()

questions = [
    "What are the key trends in sales data?",
    "Which products or categories should the business focus on?",
    "What patterns can be seen in student marks?",
    "What improvement strategies could be suggested?"
]

analysis_results = []
for q in questions:
    result = ask_openai(q, context_sales + "\n" + context_students)
    analysis_results.append({"question": q, "result": result})

# ====================================
# 9) Save Analysis to CSV
# ====================================
analysis_df = pd.DataFrame(analysis_results)
analysis_df.to_csv("analysis_results.csv", index=False)

# ====================================
# 10) Export Summary & Analysis to PDF
# ====================================
pdf_path = "analysis_report.pdf"
styles = getSampleStyleSheet()
doc = SimpleDocTemplate(pdf_path)
content = [Paragraph("Data Analysis Report", styles['Title']), Spacer(1, 12)]

content.append(Paragraph("Sales Summary", styles['Heading2']))
content.append(Paragraph(sales_summary.to_html(), styles['Normal']))

content.append(Paragraph("Students Summary", styles['Heading2']))
content.append(Paragraph(students_summary.to_html(), styles['Normal']))

content.append(Paragraph("AI Analysis Results", styles['Heading2']))
for res in analysis_results:
    content.append(Paragraph(f"Q: {res['question']}", styles['Normal']))
    content.append(Paragraph(f"A: {json.dumps(res['result'], indent=2)}", styles['Code']))

doc.build(content)

print(f"Report saved as {pdf_path} and CSV saved as analysis_results.csv")


Sales dataset shape: (960, 7)
Students dataset shape: (100, 7)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Product,960,10,Laptop,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category,960,3,Electronics,384,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Region,960,4,North,240,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Units_Sold,960.0,NaN,NaN,NaN,163.221875,80.594673,20.0,95.0,165.5,233.25,300.0
Price,960.0,NaN,NaN,NaN,253.5,243.335014,25.0,50.0,175.0,300.0,800.0
Month,960,12,January,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year,960.0,NaN,NaN,NaN,2023.5,0.500261,2023.0,2023.0,2023.5,2024.0,2024.0


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Name,100,79,Neha Khan,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Math,100.0,NaN,NaN,NaN,69.06,18.654119,36.0,53.0,70.5,84.0,100.0
Science,100.0,NaN,NaN,NaN,69.56,17.94679,36.0,53.75,70.5,85.25,100.0
English,100.0,NaN,NaN,NaN,67.77,18.423116,36.0,51.75,68.5,82.25,100.0
History,100.0,NaN,NaN,NaN,67.65,19.760881,35.0,49.0,69.5,86.0,100.0
Computer,100.0,NaN,NaN,NaN,65.9,18.249948,36.0,51.0,65.0,78.25,100.0
Average,100.0,NaN,NaN,NaN,67.988,8.387476,48.0,62.0,68.4,73.25,92.4


Combined dataset shape: (960, 14)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}